### Correcting problem with array of structures

In [ ]:
?MPI::MPI_Type_size

In [ ]:
?MPI::MPI_Type_get_extent

In [ ]:
?MPI::MPI_Type_get_true_extent

In [ ]:
?MPI::MPI_Type_create_resized

***
#### C program

In [ ]:
#include <mpi.h>

In [ ]:
%%executable  a.x -- -lmpi

int rank, size;
int right, left;
int arr_len = 5;

struct buff {
    double f;
    int i;
} snd_buf[arr_len], rcv_buf[arr_len], sum[arr_len];

int i, j;

int          array_of_blocklengths[2];
MPI_Aint     array_of_displacements[2], first_var_address, second_var_address;
MPI_Datatype array_of_types[2], send_recv_type, send_recv_resized;

MPI_Status  status;
MPI_Request request;

MPI_Init(NULL, NULL);
MPI_Comm_rank(MPI_COMM_WORLD, &rank);
MPI_Comm_size(MPI_COMM_WORLD, &size);
right = (rank+1)      % size;
left  = (rank-1+size) % size;

// Set MPI datatypes for sending and receiving partial sums
// number of elements = 1
array_of_blocklengths[0] = 1;
array_of_blocklengths[1] = 1;

MPI_Get_address(&snd_buf[0].f, &first_var_address);
MPI_Get_address(&snd_buf[0].i, &second_var_address);

array_of_displacements[0] = (MPI_Aint) 0;
array_of_displacements[1] = MPI_Aint_diff(second_var_address, first_var_address);

//datatype handles used to describe the structure
array_of_types[0] = MPI_DOUBLE;
array_of_types[1] = MPI_INT;

MPI_Type_create_struct(2, array_of_blocklengths, array_of_displacements, array_of_types, &send_recv_type);
//resize
MPI_Type_create_resized(send_recv_type, (MPI_Aint) 0, (MPI_Aint) sizeof(snd_buf[0]), &send_recv_resized);
//commit
MPI_Type_commit(&send_recv_resized);

// Init
for (j=0; j<arr_len; j++)
{
    sum[j].i = 0;
    sum[j].f = 0;
    snd_buf[j].i = (j+1)*rank;
    snd_buf[j].f = (j+1)*rank;
    rcv_buf[j].i = -1;
    rcv_buf[j].f = -1;
}

// Modify the send and receive calls to use the resized datatype
for( i = 0; i < size; i++)
{
    // to check, whether the data transfer is correct, we do not transfer the last index
    MPI_Issend(&snd_buf, arr_len-1, send_recv_resized, right, 17, MPI_COMM_WORLD, &request);
    MPI_Recv(&rcv_buf, arr_len-1, send_recv_resized, left, 17, MPI_COMM_WORLD, &status);
    MPI_Wait(&request, &status);
    for (j=0; j<arr_len; j++)
    {
        snd_buf[j] = rcv_buf[j];
        sum[j].i += rcv_buf[j].i;
        sum[j].f += rcv_buf[j].f;
    }
}

if (rank==0)
{
    printf("Expected result: for all, except the highjest j:  sum = (j+1)*(sum of all ranks)\n");
    printf("For the highest j value, no data exchange is done: sum = -(number of processes)\n");
}

for (j=0; j<arr_len; j++)
{
    printf ("PE%3i j=%3i: Sum = %6i  %8.1f\n", rank, j, (int)sum[j].i, sum[j].f);
}

MPI_Finalize();

In [ ]:
!mpirun -np 4 a.x

***
#### Fortran program

In [ ]:
%%file ring.f90
program ring
use mpi

integer, parameter :: arr_len=5
integer ( kind = 4 ) error
integer :: rank, size
integer :: i, j, right, left

type buff
    sequence
    double precision :: f
    integer :: i
end type buff
type(buff), asynchronous :: snd_buf(arr_len)
type(buff) :: rcv_buf(arr_len), sum(arr_len)

integer(kind=MPI_ADDRESS_KIND) :: first_var_address, second_var_address
integer :: send_recv_type, send_recv_resized

integer :: array_of_block_length(2)
integer :: array_of_types(2)
integer(kind=MPI_ADDRESS_KIND) :: array_of_displacements(2)
integer(kind=MPI_ADDRESS_KIND) :: lb, extent

integer :: status(MPI_STATUS_SIZE)
integer :: request

call MPI_Init(error)
call MPI_Comm_rank(MPI_COMM_WORLD, rank, error)
call MPI_Comm_size(MPI_COMM_WORLD, size, error)

right = mod(rank+1,      size)
left  = mod(rank-1+size, size)

! Create derived datatype
! number of elements = 1
array_of_block_length(1) = 1
array_of_block_length(2) = 1

! datatype handles used to describe the structure
array_of_types(1) = MPI_DOUBLE_PRECISION
array_of_types(2) = MPI_INTEGER

! get address for the first two elements
call MPI_Get_address(snd_buf(1)%f, first_var_address, error)
call MPI_Get_address(snd_buf(1)%i, second_var_address, error)

array_of_displacements(1) = 0
array_of_displacements(2) = MPI_Aint_diff(second_var_address, first_var_address)
    
call MPI_Type_create_struct(2, array_of_block_length, array_of_displacements, array_of_types, send_recv_type, error)
call MPI_Get_address(snd_buf(1), first_var_address, error)
call MPI_Get_address(snd_buf(2), second_var_address, error)
lb = 0
extent = MPI_Aint_diff(second_var_address, first_var_address)
! resize
call MPI_Type_create_resized(send_recv_type, lb, extent, send_recv_resized, error)
! commit
call MPI_Type_commit(send_recv_resized, error)
  
! Init
do j = 1, arr_len
    sum(j)%i = 0
    sum(j)%f = 0
    snd_buf(j)%i = j*rank
    snd_buf(j)%f = j*real(rank)
    rcv_buf(j)%i = -1
    rcv_buf(j)%f = -1
end do

! Modify the send and receive calls to use the resized datatype
do i = 1, size
    ! to check, whether the data transfer is correct, we do not transfer the last index
    call MPI_Issend(snd_buf, arr_len-1, send_recv_resized, right, 17, MPI_COMM_WORLD, request, error)
    call MPI_Recv(rcv_buf, arr_len-1, send_recv_resized, left, 17, MPI_COMM_WORLD, status, error)
    call MPI_Wait(request, status, error)
    if (.NOT.MPI_ASYNC_PROTECTS_NONBLOCKING) call MPI_F_sync_reg(snd_buf)
    snd_buf = rcv_buf
    do j = 1, arr_len
        sum(j)%i = sum(j)%i + rcv_buf(j)%i
        sum(j)%f = sum(j)%f + rcv_buf(j)%f
    end do
end do

if (rank == 0) then
    print *, "Expected result: for all, except the highjest j:  sum = (j+1)*(sum of all ranks)"
    print *, "For the highest j value, no data exchange is done: sum = -(number of processes)"
end if

do j = 1, arr_len
    print *,"PE", rank, " j=",j,":  Sum =", sum(j)%i, " ", real(sum(j)%f)
end do

call MPI_Finalize(error)
end

In [ ]:
!mpif90 ring.f90 && mpirun -np 4 a.out